In [1]:
# 16/3/23

he idea is to make some surface plots over the TD space to compare:
            resolution degradation for a fixed minimum AASR,
            swath portion over which RASR excedes a certain treshold,
            peak core snr
            NEXT what happens morphing the antenna size?

In [2]:
# From Michelangelo Villano's hands on lab
# To open separate plot-windows outside the browser uncomment one of the following two lines
%matplotlib qt
#get_ipython().run_line_magic('matplotlib','qt5')

# To open a Plot-window within notebook with zoom/edit control uncomment one of the following two lines
# %matplotlib notebook
# get_ipython().run_line_magic('matplotlib','notebook')

# options are 'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk' , 'notebook' , 'inline'
print("pippo")

pippo


In [3]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt
from libs.spherical_earth_geometry_radar import *
from libs.radartools.farField import UniformAperture
from libs.ambiguity_functions import *
from unambiguous_imaging_design_analysis import *
import matplotlib as mpl

_*Problem : how to sample the space:*_

**possibility 1** we restrict the sampling space to solutions which align at the far range (transmit event excluded) with the next nadir return.
- few points have to be computed
- some dynamics might be hidden

**possibility 2** we set a minimum and maximum range, we discretize the PRF axis, and we find all possible ranges for each sample within the swath.
- resolution set by the user
- the samples might not coincide with the nadir return avoidance optimal (a visual selection can still be made in any case).

**2** is better overall

Also we can use the intersection point of the two minprf and opt prf curves as a reference point, but how?

In [4]:
## System Parameters

# constants
c = 299792458

# radar parameters
dutycycle = 0.25  # duty cycle
h = 500e3  # height
wavelength = c / 10e9  # wavelength

# satellite speed
vs = orbital_speed(h)

# antenna size
Wa = 0.3  # antenna width in meters
La = 2  # antenna length in meters

# nadir duration in fractions of PRI for visualization in timing diagram
# note, it makes sense to use this fractional quantity as the nadir duration, if unfocused (e.g. saturated receiver), will be proportional to the impulse on time.
nadir_duration = 2 * dutycycle


In [5]:
## base diagram
# PRF axis
PRI = 1 / 7050
prf = np.linspace(1 / PRI - 1500, 1 / PRI + 10000, 100)

# plot
fig, ax = plt.subplots(1, dpi=150)
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False)
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h)
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)

C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


(100.0, 300.0)

In [6]:
## canonical constraints line plots
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000

prf_opt = optimal_prf_line(ground_range, Wa, wavelength, h)
# Add to the plot
ax.plot(prf_opt, ground_range / 1000, 'red')

prf_min = minimum_prf_line(ground_range, vs, La, h)
ax.plot(prf_min, ground_range / 1000, 'green')

C:\Users\smen851\PycharmProjects\systemtradeoffs\unambiguous_imaging_design_analysis.py:56: RuntimeWarning: divide by zero encountered in divide
  prf_opt = 1 / pri


In [7]:
# find the intersection point
tosolve = lambda rgx: optimal_prf_line(rgx, Wa, wavelength, h) - minimum_prf_line(rgx, vs, La, h)

rg_point = fsolve(tosolve, 200000)
prf_point = minimum_prf_line(rg_point, vs, La, h)

print(prf_point, rg_point)

ax.plot(prf_point, rg_point / 1000, 'x', color='blue')

[7058.91598277] [197427.8613883]


At this point we have the timing diagram, the minimum PRF line, the maximum range line, and the intersetion point representing the ideal design solution.

The intersection point will be the center sample for the analysis; the extent is set empirically.

# Phase0: defining the meshgrid for calculation

In [8]:
# number of PRF samples
x_samples = 200
# number of ground range samples
y_samples = 3

prf_axis = np.linspace(prf_point - 1e3, prf_point + 10e3, x_samples)
prf_mesh = np.repeat(prf_axis, y_samples).reshape(x_samples, y_samples)
#print(prf_mesh)

# ground range mesh coordinates initialization
rg_mesh = np.zeros_like(prf_mesh)
for ii in range(x_samples):
    prf_point = prf_axis[ii]
    # the center range sample shall be on the red curve
    rg_opt = lambda rg: optimal_prf_line(rg, Wa, wavelength, h) - prf_point
    rg_point = fsolve(rg_opt, rg_point)  # todo red curve
    # 1 convert rg to slant range
    rs = range_ground_to_slant(rg_point, h)
    # 2 find the next pulse order
    n = np.ceil((2 * rs / c) * prf_point)
    # 3 find the nadir order
    n_nadir = np.ceil((2 * h / c) * prf_point)
    # 4 generate the range axis
    if n - n_nadir <= y_samples / 2:  # to maintain a side looking geometry and the same number of samples
        rs_axis = (np.arange(n_nadir, n_nadir + y_samples) + 0.5) * c / (2 * prf_point)
        rg_axis, tt = range_slant_to_ground(rs_axis, h)
    else:
        rs_axis = (np.arange(n - int(y_samples / 2) - 1, n - int(y_samples / 2) - 1 + y_samples) + 0.5) * c / (
                2 * prf_point)
        rg_axis, tt = range_slant_to_ground(rs_axis, h)
    rg_mesh[ii] = rg_axis

# it seems to work. tested with different values of x samples and y samples

In [9]:
print(prf_mesh)
print(rg_mesh)

[[ 6058.91598277  6058.91598277  6058.91598277]
 [ 6114.19236468  6114.19236468  6114.19236468]
 [ 6169.46874658  6169.46874658  6169.46874658]
 [ 6224.74512849  6224.74512849  6224.74512849]
 [ 6280.0215104   6280.0215104   6280.0215104 ]
 [ 6335.29789231  6335.29789231  6335.29789231]
 [ 6390.57427422  6390.57427422  6390.57427422]
 [ 6445.85065613  6445.85065613  6445.85065613]
 [ 6501.12703804  6501.12703804  6501.12703804]
 [ 6556.40341995  6556.40341995  6556.40341995]
 [ 6611.67980186  6611.67980186  6611.67980186]
 [ 6666.95618377  6666.95618377  6666.95618377]
 [ 6722.23256568  6722.23256568  6722.23256568]
 [ 6777.50894759  6777.50894759  6777.50894759]
 [ 6832.7853295   6832.7853295   6832.7853295 ]
 [ 6888.06171141  6888.06171141  6888.06171141]
 [ 6943.33809332  6943.33809332  6943.33809332]
 [ 6998.61447523  6998.61447523  6998.61447523]
 [ 7053.89085714  7053.89085714  7053.89085714]
 [ 7109.16723905  7109.16723905  7109.16723905]
 [ 7164.44362096  7164.44362096  7164.44

In [10]:
rg_mesh.shape

(200, 3)

In [11]:
# visualization
# plotting the points in the already open timing diagram.

X, Y = prf_mesh, rg_mesh

for ii in range(x_samples * y_samples):
    ax.plot(X.flatten()[ii], Y.flatten()[ii] / 1000, 'x', color='green')

# adjust the plot scale
ax.set_xlim(X.min() - 3e2, X.max() + 3e2)
ax.set_ylim(Y.min() / 1e3 - 30, Y.max() / 1e3 + 30);

# Phase1: Design optimization for every point of the grid

In [12]:
# maximum acceptable AASR level
AASR_max = -20  #dB
# maximum acceptable RASR level
RASR_max = -20  #dB

In [13]:
##  Initialization of the geometry model and aperture model

# Radar Geometry object
radar_geo = RadarGeometry()
radar_geo.set_rotation(30 * np.pi / 180, 0, 0)
radar_geo.set_initial_position(0, 0, h)
radar_geo.set_speed(vs)
# no need to set the radar rotation as this will be done inside the analysys function

# Uniform aperture antenna initialization
uniap = UniformAperture(La, Wa, c / wavelength)

In [ ]:
# optimize the design for every point in the grid and store the results in a list
# also directly store the results of interest in separate lists
results_list = []
x_coordinate = []  # prf coordinate
y_coordinate = []  # optimized ground range coordinate (compressed image ground swath central point)
undersampling_list = []  # optimized undersampling ratio to limit AASR
rasr_free_swath_list = []  # portion of the ground range free from ambiguities
core_snr_list = []
for ii in range(x_samples):
    for jj in range(y_samples):
        dictionary = umambiguous_mode_analysis(radar_geo,
                                               uniap, rg_mesh[ii, jj],
                                               prf_mesh[ii, jj], AASR_max, RASR_max,
                                               vs, wavelength, dutycycle,
                                               aasr_points=15, rasr_points=25)
        # saving all the analysis results
        results_list.append(dictionary)
        # saving individual relevant figures from the analysis
        x_coordinate.append(np.average(dictionary['prf']))
        y_coordinate.append(
            np.average(dictionary['compressed-ground-swath']))  # this is the mid-point of the usable ground swath
        undersampling_list.append(dictionary['minimum-undersampling'])
        rasr_free_swath_list.append(dictionary['usable-rasr-swath'])
        core_snr_list.append(dictionary['core-snr-linear'])

optimized looking angle: 19.23830579294688


100%|██████████| 15/15 [00:32<00:00,  2.19s/it]


optimized looking angle: 25.14454768539982


100%|██████████| 15/15 [00:34<00:00,  2.32s/it]


optimized looking angle: 29.550811022428967


...

100%|██████████| 15/15 [00:30<00:00,  2.00s/it]


optimized looking angle: 27.233273175497114


100%|██████████| 15/15 [00:29<00:00,  1.98s/it]


optimized looking angle: 15.607439070913026


 80%|████████  | 12/15 [00:23<00:05,  1.96s/it]

Note return dictionary of unambiguous_mode_analysis
    return_dict = {
        'ground-swath': swath_rg,
        'prf': swath_prf,
        'compressed-ground-swath': swath_rg_compressed,
        'compressed-slant-swath': swath_rs_compressed,
        'compressed-incidence-swath': swath_eta_compressed,
        'rasr-ground-range-axis': ground_range_axis,
        'rasr-slant-range-axis': slant_range_axis,
        'rasr-incidence-axis': incidence_axis,
        'rasr-linear': rasr,
        'rasr-doppler-bandwidth': Bd,
        'aasr-undersampling-axis': doppler_undersampling_ratio,
        'aasr-linear': aasr,
        'core-snr-linear': core_snr,
        'corrected-azimuth-resolution': azres,
        'minimum-undersampling': undersampling,
        'usable-rasr-swath': swath_rasr,
        'usable-swath-ranges': ranges,
        'optimized-looking-angle': looking_angle_opt[0]
    }

In [42]:
# test
print(x_coordinate)

[6058.915982765739, 6058.915982765739, 6058.915982765739, 6114.192364675287, 6114.192364675287, 6114.192364675287, 6169.468746584834, 6169.468746584834, 6169.468746584834, 6224.745128494382, 6224.745128494382, 6224.745128494382, 6280.02151040393, 6280.02151040393, 6280.02151040393, 6335.297892313477, 6335.297892313477, 6335.297892313477, 6390.574274223025, 6390.574274223025, 6390.574274223025, 6445.850656132573, 6445.850656132573, 6445.850656132573, 6501.12703804212, 6501.12703804212, 6501.12703804212, 6556.403419951668, 6556.403419951668, 6556.403419951668, 6611.679801861216, 6611.679801861216, 6611.679801861216, 6666.956183770763, 6666.956183770763, 6666.956183770763, 6722.232565680311, 6722.232565680311, 6722.232565680311, 6777.508947589859, 6777.508947589859, 6777.508947589859, 6832.7853294994065, 6832.7853294994065, 6832.7853294994065, 6888.061711408955, 6888.061711408955, 6888.061711408955, 6943.338093318503, 6943.338093318503, 6943.338093318503, 6998.61447522805, 6998.6144752280

# Phase1.5: results saver and loader for asynchronous operation
i.e. a cell to pickle the list and a cell to unpickle the list.


### SAVER
run the cell below to save the analysis results in a file:
- use pickle to
- it shall create a folder called analysis_results if not existing yet in the working folder
- it shall save as individual files
        > results_list
        > x_coordinate
        > y_coordinate
        > undersampling_list
        > rasr_free_swath_list
        > core_snr_list
    keeping the same naming convention for the files as the content list. e.g. ./analysis_results/results_list.pk <-- results_list

In [43]:
import os
import pickle

# Create a folder for the analysis results if it does not exist
if not os.path.exists('analysis_results'):
    os.makedirs('analysis_results')

# Save the results_list
with open('analysis_results/results_list.pk', 'wb') as f:
    pickle.dump(results_list, f)
# Save the x_coordinate
with open('analysis_results/x_coordinate.pk', 'wb') as f:
    pickle.dump(x_coordinate, f)
# Save the y_coordinate
with open('analysis_results/y_coordinate.pk', 'wb') as f:
    pickle.dump(y_coordinate, f)
# Save the undersampling_list
with open('analysis_results/undersampling_list.pk', 'wb') as f:
    pickle.dump(undersampling_list, f)
# Save the rasr_free_swath_list
with open('analysis_results/rasr_free_swath_list.pk', 'wb') as f:
    pickle.dump(rasr_free_swath_list, f)
# Save the core_snr_list
with open('analysis_results/core_snr_list.pk', 'wb') as f:
    pickle.dump(core_snr_list, f)


ChatGpt: Sure! Here's the specification for loading the analysis results from the saved pickle files:

### LOADER
- Load the analysis results from the saved pickle files using pickle module.
- Load the following individual files from the "analysis_results" folder:
    * results_list.pk
    * x_coordinate.pk
    * y_coordinate.pk
    * undersampling_list.pk
    * rasr_free_swath_list.pk
    * core_snr_list.pk
- Assign the loaded data to the corresponding variable names:
    * results_list
    * x_coordinate
    * y_coordinate
    * undersampling_list
    * rasr_free_swath_list
    * core_snr_list

Note: The file names and variable names must match with the ones used in the SAVER section.
Note1: Run the cells preceding and in Phase 0 before or after loading

In [44]:
import pickle
from IPython.display import Javascript

Javascript("Jupyter.notebook.execute_cell_range(1,11)")  #doesn't work. run clls manually up to phase0

# Load the results_list
with open('analysis_results/results_list.pk', 'rb') as f:
    results_list = pickle.load(f)
# Load the x_coordinate
with open('analysis_results/x_coordinate.pk', 'rb') as f:
    x_coordinate = pickle.load(f)
# Load the y_coordinate
with open('analysis_results/y_coordinate.pk', 'rb') as f:
    y_coordinate = pickle.load(f)
# Load the undersampling_list
with open('analysis_results/undersampling_list.pk', 'rb') as f:
    undersampling_list = pickle.load(f)
# Load the rasr_free_swath_list
with open('analysis_results/rasr_free_swath_list.pk', 'rb') as f:
    rasr_free_swath_list = pickle.load(f)
# Load the core_snr_list
with open('analysis_results/core_snr_list.pk', 'rb') as f:
    core_snr_list = pickle.load(f)

# Phase2: grid visualization

In [45]:
# plotting the points in the already open timing diagram.


for ii in range(len(x_coordinate)):
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 30, max(y_coordinate) / 1e3 + 30)


(33.70160275871954, 321.89876941850025)

## Phase2.5: colorbar visualization tool for design solutions
Requirements
- vertical lines corresponding to the usable (RASR free)swath
- color of lines corresponding to resolution or core snr


In [46]:
# prototype
def design_point_visualizer(axis, dictionary, scalar_element, colormap, scalar_min, scalar_max,
                            labeling=True,
                            cmapaxis=0,
                            scale=1,
                            unit=' ',
                            logscale=False, h=500e3,
                            re=6371e3):
    """

    :param axis: matplotlib axis for the plot
    :param dictionary: dictionary from analysis
    :param scalar_element: a string containing the dictionary key pointing to a scalar value
    :param colormap: matplotlib colormap key
    :param scalar_min: minimum scalar value for the colorbar, linear
    :param scalar_max: maximum scalar value for the colorbar, linear
    :param labeling: default True, adds a label to every line in the plot
    :param cmapaxis: axis for the colormap, if not set, no colormap is produced
    :param scale: multiplier for the scalar value
    :param unit: string to display on the label after the number
    :param logscale: if set to true, the scalar value is converted to decibel
    :return: stocazzo
    """
    #1 find holes in the usable swath ranges, note the swath ranges are linearly spaced over the incidence angle axis
    ranges = dictionary['usable-swath-ranges']
    slant_ranges = range_ground_to_slant(ranges, h, re)
    ranges1, incidences = range_slant_to_ground(slant_ranges, h, re)
    deltas = incidences[1:-1] - incidences[0:-2]  # so far so good
    if deltas.size != 0:
        delta = min(deltas)
    else:
        delta = 0
    #print(delta)
    holes = np.argwhere(deltas > 1.01 * delta)  #contains the indexes of the ranges element preceding a hole
    #2 color code for scalar value
    cmap = matplotlib.colormaps[colormap]  # todo change to colormap
    scalar = np.average(dictionary[scalar_element]) * scale
    if logscale:
        scalar = 10 * np.log10(scalar)
        scalar_min = 10 * np.log10(scalar_min)
        scalar_max = 10 * np.log10(scalar_max)
    mapped_color = cmap((scalar - scalar_min) / (scalar_max - scalar_min))
    #3 draw a line for every contiguous section of range
    if holes.size == 0:
        holes = np.array([-1])
    else:
        holes += 1
        holes = np.append(holes, -1)
    holes = np.insert(holes, 0, 0)
    #print(deltas)
    #print(holes)
    for ii in range(holes.size - 1):
        yy = ranges[holes[ii]: holes[ii + 1]]
        xx = np.ones_like(yy) * np.average(dictionary['prf'])
        label = str(round(scalar, 2)) + unit
        axis.plot(xx, yy / 1000, color=mapped_color, label=label, linewidth=2.5)
        if labeling:
            axis.annotate(label, (np.average(xx) + 2, np.average(yy) / 1000))

    # plot a colored dot if the range is zero
    if ranges.size == 0:
        label = str(round(scalar, 2)) + unit
        axis.plot(np.average(dictionary['prf']), np.average(dictionary['compressed-ground-swath']) / 1000, '.',
                  color=mapped_color, label=label, linewidth=2.5)
        if labeling:
            axis.annotate(label, (np.average(dictionary['prf']) + 2,
                                  np.average(dictionary['compressed-ground-swath']) / 1000))

    if cmapaxis != 0:
        # colormap
        #cmap = plt.cm.get_cmap(colormap) #deprecated
        cmap = mpl.colormaps[colormap]
        norm = mpl.colors.Normalize(vmin=scalar_min, vmax=scalar_max)
        cb1 = mpl.colorbar.ColorbarBase(cmapaxis, cmap=cmap,
                                        norm=norm,
                                        orientation='horizontal')
        cb1.set_label(unit)
    #plt.show()

# dictionary = results_list[3]
# design_point_visualizer(ax, dictionary, 'minimum-undersampling', 'Greys', 0, 1, logscale=False, h=h)

In [47]:
# test
for dictionary in results_list:
    design_point_visualizer(ax, dictionary, 'minimum-undersampling', 'plasma', 0, 1, logscale=False, h=h)

# Phase3: Ground range visualization
Requirements:
- the timing diagram shall be black and white
- the X in the plot shall show the position of the radar midswath on ground
- the color shall be representative of the total ground swath in km.

In [48]:
# 1 create another timing diagram in black and white:
# plot
fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('RASR-free Ground range visualization')
fig.tight_layout()

In [49]:
# 2 midswath points:
for ii in range(len(x_coordinate)):
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 50, max(y_coordinate) / 1e3 + 50)

(13.701602758719538, 341.89876941850025)

In [50]:
# colored lines:
for dictionary in results_list:
    design_point_visualizer(ax, dictionary, 'usable-rasr-swath', 'plasma', 0, 60, scale=1e-3, unit='km',
                            logscale=False, h=h, cmapaxis=ax1, labeling=False)
fig.tight_layout()

In [51]:
#mpl.colormaps['plasma']

# Phase4: undersampling/resolution visualization

In [52]:
# 1 create another timing diagram in black and white:
# plot
fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('Doppler under-processing ratio visualization')
fig.tight_layout()

In [53]:
# 2 midswath points:
for ii in range(len(x_coordinate)):
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 50, max(y_coordinate) / 1e3 + 50)

(13.701602758719538, 341.89876941850025)

In [54]:
# colored lines:
for dictionary in results_list:
    design_point_visualizer(ax, dictionary, 'minimum-undersampling', 'plasma', 0, 1,
                            logscale=False, h=h, cmapaxis=ax1, labeling=False)
fig.tight_layout()

# Phase5: CoreSNR visualization

In [55]:
# 1 create another timing diagram in black and white:
# plot
fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('Core SNR visualization')
fig.tight_layout()

In [56]:
# 2 midswath points:
for ii in range(len(x_coordinate)):
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 50, max(y_coordinate) / 1e3 + 50)

(13.701602758719538, 341.89876941850025)

In [57]:
# colored lines:
for dictionary in results_list:
    design_point_visualizer(ax, dictionary, 'core-snr-linear', 'plasma', 10 ** 9, 10 ** 10.2, unit='dB',
                            logscale=True, h=h, cmapaxis=ax1, labeling=False)
fig.tight_layout()

## Phase 5.5 Core SNR considering minimum Doppler underprocessing ratio

In [58]:
# 1 create another timing diagram in black and white:
# plot
fig, (ax, ax1) = plt.subplots(2, dpi=150, gridspec_kw={'height_ratios': [30, 1]})
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False, color='dimgray')
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h, color='silver')
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)
## canonical constraints line plots
ax.plot(prf_opt, ground_range / 1000, 'k')
ax.plot(prf_min, ground_range / 1000, '--k')
ax.set_title('Underprocessed Doppler Core SNR visualization')
fig.tight_layout()

In [59]:
# 2 midswath points:
for ii in range(len(x_coordinate)):
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 50, max(y_coordinate) / 1e3 + 50)

(13.701602758719538, 341.89876941850025)

In [60]:
# colored lines:
for dictionary in results_list:
    design_point_visualizer(ax, dictionary, 'core-snr-linear-underprocessed', 'plasma', 10 ** 9, 10 ** 10.2, unit='dB',
                            logscale=True, h=h, cmapaxis=ax1, labeling=False)
fig.tight_layout()

# Pase6: individual point detailed visualization
- get the coordinates from click on plot ( not doable in jupyter )
- find the closest point in the solution list
- draw a 2x2 multiplot with AASR curve, RASR curve, SNR core, SNR core with doppler underprocessing

In [61]:
# event_x = 0
# event_y = 0
# def onclick(event):
#     print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
#           ('double' if event.dblclick else 'single', event.button,
#            event.x, event.y, event.xdata, event.ydata))
#     event_x, event_y = event.x, event.y
# # connect onclick to the figure
# cid = fig.canvas.mpl_connect('button_press_event', onclick)
# plt.show()

Set here X and Y approximate coordinate in Hz, Km

In [62]:
hline, vline = 'a', 'b'

In [63]:
x = 8110     # Hz
y = 160  # km

In [64]:
# look in x, y array
# we exploit the fact that the range samples vary faster than the prf ones (they're in the inner loop for generation)
# first we find the closest prf index
start_x_index = int(np.floor((x - min(x_coordinate)) * x_samples / (max(x_coordinate) - min(x_coordinate))) * y_samples)
print(start_x_index)
# then we find the closest range via binary search
index = np.argmin(np.abs(np.array(y_coordinate[start_x_index:start_x_index + y_samples]) - y * 1000)) + start_x_index
print(index)

111
111


In [65]:

# highlight the point in the plot

if hline != 'a':
    print('hello')
    line = hline.pop(0)
    line.remove()
    line = vline.pop(0)
    line.remove()

hline = ax.plot(np.array([min(x_coordinate) - 3e2, max(x_coordinate) + 3e2]), np.ones(2) * y_coordinate[index] / 1e3,
                'r',linewidth=1)
vline = ax.plot(np.ones(2) * x_coordinate[index],
                np.array([min(y_coordinate) / 1e3 - 50, max(y_coordinate) / 1e3 + 50]), 'r',linewidth=1)
plt.show()
print('helllo')

helllo


quadruple plot

In [66]:
# maximum acceptable AASR level
AASR_max = -20  #dB
# maximum acceptable RASR level
RASR_max = -20  #dB

fig1, axs = plt.subplots(2, dpi=150)
dictionary = results_list[index]
# rasr
rasr = dictionary['rasr-linear']
ground_range_axis = dictionary['rasr-ground-range-axis']
axs[0].plot(ground_range_axis / 1000, 10 * np.log10(rasr))
axs[0].set_xlabel('ground range [km]')
axs[0].set_ylabel('RASR [dB]')
 # add to plot the RASR threshold
swath_rasr = dictionary['usable-rasr-swath']
ranges = dictionary['usable-swath-ranges']
if swath_rasr != 0:
    axs[0].scatter(ranges / 1000, np.ones_like(ranges) * RASR_max, marker='.', color='r',
                 label=str('rg=' + str(np.round(swath_rasr[0] / 1000, 2)) + '[km]'))
    axs[0].legend()
# aasr
aasr = dictionary['aasr-linear']
doppler_undersampling_ratio = dictionary['aasr-undersampling-axis']
axs[1].plot(doppler_undersampling_ratio, 10 * np.log10(aasr), '*-')
axs[1].set_xlabel('Doppler undersampling ratio')
axs[1].set_ylabel('AASR [dB]')
# AASR undersampling point (In 25)
undersampling = dictionary['minimum-undersampling']
axs[1].scatter(undersampling, AASR_max, marker='o', color='r',
             label='undersampling ratio =' + str(np.round(undersampling, 2)))

fig1.tight_layout()
plt.show()

In [67]:
print(axs)

[<Axes: xlabel='ground range [km]', ylabel='RASR [dB]'>
 <Axes: xlabel='Doppler undersampling ratio', ylabel='AASR [dB]'>]


In [68]:
print(dictionary)

{'ground-swath': array([138260.26961824, 180771.47139324]), 'prf': array([8104.14211342, 8104.14211342]), 'compressed-ground-swath': array([146121.41807886, 174336.12087795]), 'compressed-slant-swath': array([522519.0291565 , 531767.15356634]), 'compressed-incidence-swath': array([0.30633736, 0.36134015]), 'rasr-ground-range-axis': array([146121.41807886, 147278.6530733 , 148437.38426542, 149597.62522   ,
       150759.38956104, 151922.69097256, 153087.54319914, 154253.96004645,
       155421.95538198, 156591.54313582, 157762.73730101, 158935.55193454,
       160110.00115763, 161286.09915679, 162463.86018414, 163643.29855832,
       164824.42866505, 166007.26495793, 167191.82195893, 168378.11425931,
       169566.15652028, 170755.96347347, 171947.54992194, 173140.93074086,
       174336.12087795]), 'rasr-slant-range-axis': array([522519.0291565 , 522869.27912015, 523222.512013  , 523578.73667599,
       523937.96203666, 524300.19710962, 524665.45099707, 525033.73288937,
       525405.0